# 📊 02. Feature Engineering Analysis

**Lab 02: Parsing & Reference Matching**  
**MSSV:** 23120067 - Lê Minh Nhật  
**Mục tiêu:** Phân tích và justify các features cho Reference Matching (Yêu cầu 2.2.3)

---

## 📋 Nội dung Notebook

1. [Setup & Import](#1-setup--import)
2. [Data Preparation](#2-data-preparation)
3. [Feature Groups Analysis](#3-feature-groups-analysis)
4. [Feature Correlation Analysis](#4-feature-correlation-analysis)
5. [Feature Importance Justification](#5-feature-importance-justification)
6. [Summary & Conclusions](#6-summary--conclusions)

---

## Yêu cầu từ text2.txt (Section 2.2.3)

> "Students must perform feature engineering to construct reasonable features for the model. 
> Students must **justify the creation of each feature** in the final report, explaining the underlying idea."

### Feature Groups (19+ features):
| Group | Features | Purpose |
|-------|----------|---------|
| Title | 5 | Primary matching signal |
| Author | 5 | Identity verification |
| Year | 4 | Temporal filtering |
| Text | 5 | Deep content matching |
| Hierarchy | 5+ | Citation context |

## 1. Setup & Import

In [ ]:
# ============================================
# 1.1 Import thư viện
# ============================================
import sys
import os
from pathlib import Path
import json
import re
from collections import Counter

# Increase recursion limit
sys.setrecursionlimit(10000)

# Disable pyparsing packrat
try:
    import pyparsing
    pyparsing.ParserElement.disablePackrat()
except (ImportError, AttributeError):
    pass

# Data analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text processing
from fuzzywuzzy import fuzz

# Setup paths
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / 'src'
OUTPUT_DIR = PROJECT_ROOT / 'output'

# Add src to path
sys.path.insert(0, str(SRC_DIR))

# Import project modules
from matcher.feature_extractor import FeatureExtractor
from matcher.hierarchy_features import HierarchyFeatureExtractor
from matcher.data_preparation import DataPreparator
from utils.file_io import load_json

print(f"✅ Project root: {PROJECT_ROOT}")
print(f"✅ Output directory: {OUTPUT_DIR}")